# Results p. 1
## Sample-by-Sample Evaluation 

In [21]:
import copy

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from analysis._article_results.hfc._helpers import *
import analysis.statistics.sample_metrics as sm

# pio.renderers.default = "browser"

## Load Data

In [22]:
global_measures = sm.load_global_metrics(DATASET_NAME, PROCESSED_DATA_DIR, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
global_measures.drop(index=[peyes.constants.ACCURACY_STR, peyes.constants.BALANCED_ACCURACY_STR], inplace=True)    # Drop Acc+Balanced-Acc metrics

# drop human annotators that aren't GT1 or GT2 (and aren't detectors)
annotators_to_drop = [ann for ann in global_measures.columns.get_level_values(u.PRED_STR).unique() if ann not in [GT1, GT2] and ann not in DETECTORS.keys()]
global_measures.drop(columns=annotators_to_drop, level=u.GT_STR, inplace=True)
global_measures.drop(columns=annotators_to_drop, level=u.PRED_STR, inplace=True)

global_measures

trial_id              1                                                     \
gt                    MN                                                     
pred                idvt        nh        RA      ivvt  remodnav   engbert   
cohen's_kappa   0.236827  0.345449  0.789915  0.006621  0.018960  0.357025   
complement_nld  0.799422  0.820524  0.941582  0.059529  0.110395  0.802310   
mcc             0.255316  0.374803  0.791450  0.018534  0.060351  0.382572   

trial_id                                                ...        10  \
gt                                        RA            ...        MN   
pred                 ivt       idt      idvt        nh  ...       ivt   
cohen's_kappa   0.034027  0.141882  0.273426  0.329668  ...  0.035190   
complement_nld  0.199689  0.800311  0.796535  0.807641  ...  0.227575   
mcc             0.087158  0.160184  0.299500  0.362199  ...  0.091059   

trial_id                                                                    \
gt                              RA                                           
pred                 idt      idvt        nh      ivvt  remodnav   engbert   
cohen's_kappa   0.200674  0.204895  0.363214  0.004242  0.019474  0.365619   
complement_nld  0.837034  0.843917  0.827487  0.074378  0.150311  0.829485   
mcc             0.250508  0.251479  0.392835  0.012381  0.057021  0.396546   

trial_id                                      
gt                                            
pred                  MN       ivt       idt  
cohen's_kappa   0.927445  0.036265  0.204895  
complement_nld  0.981572  0.229352  0.843917  
mcc             0.927873  0.095600  0.251479  

[3 rows x 160 columns]

In [23]:
fixation_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=1, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
fixation_sdt = fixation_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]      # Keep only d' and f1 metrics
fixation_sdt = fixation_sdt.rename(index=lambda idx: f"fixation_{idx}")     # Rename index

# drop human annotators that aren't GT1 or GT2 (and aren't detectors)
annotators_to_drop = [ann for ann in fixation_sdt.columns.get_level_values(u.PRED_STR).unique() if ann not in [GT1, GT2] and ann not in DETECTORS.keys()]
fixation_sdt.drop(columns=annotators_to_drop, level=u.GT_STR, inplace=True)
fixation_sdt.drop(columns=annotators_to_drop, level=u.PRED_STR, inplace=True)

fixation_sdt

trial_id                1                                                     \
gt                      MN                                                     
pred                  idvt        nh        RA      ivvt  remodnav   engbert   
fixation_d_prime  1.632837  2.588093  2.810875  1.257979  0.430529  2.367849   
fixation_f1       0.927405  0.962072  0.963768  0.120229  0.212726  0.949652   

trial_id                                                  ...        10  \
gt                                          RA            ...        MN   
pred                   ivt       idt      idvt        nh  ...       ivt   
fixation_d_prime  0.960429  1.047735  1.958827  2.487490  ...  0.961187   
fixation_f1       0.377801  0.908329  0.935551  0.954533  ...  0.418768   

trial_id                                                                      \
gt                                RA                                           
pred                   idt      idvt        nh      ivvt  remodnav   engbert   
fixation_d_prime  2.177725  2.102245  2.729531  0.666834  0.762996  2.808797   
fixation_f1       0.936413  0.939911  0.966601  0.146966  0.297901  0.970138   

trial_id                                        
gt                                              
pred                    MN       ivt       idt  
fixation_d_prime  3.937406  1.105628  2.102245  
fixation_f1       0.988690  0.420599  0.939911  

[2 rows x 160 columns]

## Statistical Analyses
### Sample-by-Sample Agreement
Evaluate performance on the sample-by-sample level using metrics for all labels together:
- Cohen's $\kappa$
- MCC
- $1-NLD$ 

In [24]:
global_stats, global_pvalues, global_nemenyi, sm_global_Ns = sm.friedman_nemenyi(global_measures, [GT1, GT2])

pd.concat([global_stats, global_pvalues, global_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)

Q             p  is_sig
metric         gt                                 
cohen's_kappa  MN  57.145422  1.707174e-10    True
               RA  57.576302  1.396404e-10    True
complement_nld MN  49.842011  5.056668e-09    True
               RA  49.174147  6.880875e-09    True
mcc            MN  56.989247  1.836102e-10    True
               RA  57.763441  1.279663e-10    True

#### Post-Hoc Analysis
**Cohen's Kappa**

In [25]:
post_hoc_kappa = sm.post_hoc_table(global_nemenyi, peyes.constants.COHENS_KAPPA_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_kappa

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.      **       *     n.s.
         RA      --    n.s.    n.s.    n.s.       *       *     n.s.
ivvt     MN  0.8023      --       †       *     ***     ***     n.s.
         RA  0.8166      --       †       *     ***     ***     n.s.
idt      MN  0.8413  0.0724      --    n.s.    n.s.    n.s.     n.s.
         RA  0.8095  0.0650      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.5187  0.0126  0.9989      --    n.s.    n.s.     n.s.
         RA  0.5124  0.0135  0.9994      --    n.s.    n.s.     n.s.
engbert  MN  0.0054  0.0000  0.3278  0.6738      --    n.s.      ***
         RA  0.0120  0.0000  0.4998  0.7998      --    n.s.      ***
nh       MN  0.0120  0.0000  0.4563  0.7949  1.0000      --       **
         RA  0.0283  0.0000  0.6618  0.9052  1.0000      --       **
remodnav MN  0.9992  0.9699  0.5375  0.2200  0.0006  0.0015       --
         RA  0.9928  0.9945  0.3388  0.1226  0.0005  0.0015       --

In [26]:
post_hoc_mcc = sm.post_hoc_table(global_nemenyi, peyes.constants.MCC_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_mcc

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.      **       *     n.s.
         RA      --    n.s.    n.s.    n.s.       *       *     n.s.
ivvt     MN  0.7061      --       †       *     ***     ***     n.s.
         RA  0.7399      --       †       *     ***     ***     n.s.
idt      MN  0.8681  0.0528      --    n.s.    n.s.    n.s.     n.s.
         RA  0.8478  0.0544      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.5910  0.0101  0.9994      --    n.s.    n.s.     n.s.
         RA  0.5596  0.0104  0.9994      --    n.s.    n.s.     n.s.
engbert  MN  0.0075  0.0000  0.3416  0.6588      --    n.s.      ***
         RA  0.0124  0.0000  0.4532  0.7669      --    n.s.      ***
nh       MN  0.0153  0.0000  0.4594  0.7721  1.0000      --       **
         RA  0.0362  0.0000  0.6588  0.9068  1.0000      --       **
remodnav MN  0.9988  0.9444  0.5533  0.2540  0.0007  0.0017       --
         RA  0.9949  0.9810  0.4168  0.1628  0.0006  0.0026       --

In [27]:
post_hoc_nld = sm.post_hoc_table(global_nemenyi, peyes.constants.COMPLEMENT_NLD_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_nld

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
ivvt     MN  0.8258      --     ***      **      **     ***     n.s.
         RA  0.8212      --      **      **      **      **     n.s.
idt      MN  0.1541  0.0009      --    n.s.    n.s.    n.s.        *
         RA  0.1793  0.0012      --    n.s.    n.s.    n.s.        *
idvt     MN  0.1718  0.0012  1.0000      --    n.s.    n.s.        *
         RA  0.1646  0.0010  1.0000      --    n.s.    n.s.        *
engbert  MN  0.2783  0.0033  1.0000  1.0000      --    n.s.        *
         RA  0.2540  0.0026  1.0000  1.0000      --    n.s.        *
nh       MN  0.1393  0.0008  1.0000  1.0000  0.9999      --        *
         RA  0.1812  0.0013  1.0000  1.0000  1.0000      --        *
remodnav MN  0.9912  0.9963  0.0150  0.0177  0.0384  0.0129       --
         RA  0.9856  0.9979  0.0145  0.0126  0.0258  0.0147       --

### Fixation Sample-by-Sample Detection
Evaluate performance by measuring how well the detector detects _fixation-samples_ out of all samples.
Evaluation is based on _Discriminability Index_ ($d'$) and _f1-score_

In [28]:
sdt_statistics, sdt_pvalues, sdt_nemenyi, sdt_Ns = sm.friedman_nemenyi(fixation_sdt, [GT1, GT2])

pd.concat([sdt_statistics, sdt_pvalues, sdt_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)

Q             p  is_sig
metric           gt                                 
fixation_d_prime MN  50.848921  3.176149e-09    True
                 RA  55.467626  3.729411e-10    True
fixation_f1      MN  57.323741  1.570971e-10    True
                 RA  57.280576  1.602913e-10    True

#### Post Hoc Analysis
**_d'_**

In [29]:
post_hoc_fix_dprime = sm.post_hoc_table(sdt_nemenyi, f"fixation_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_fix_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.      **      **     n.s.
         RA      --    n.s.    n.s.    n.s.       *       *     n.s.
ivvt     MN  0.9988      --    n.s.    n.s.     ***     ***     n.s.
         RA  1.0000      --    n.s.    n.s.      **       *     n.s.
idt      MN  0.6886  0.3360      --    n.s.    n.s.    n.s.     n.s.
         RA  0.7668  0.6768      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.4594  0.1664  0.9999      --    n.s.    n.s.     n.s.
         RA  0.4349  0.3415  0.9991      --    n.s.    n.s.        †
engbert  MN  0.0048  0.0004  0.4842  0.7118      --    n.s.       **
         RA  0.0116  0.0065  0.5470  0.8520      --    n.s.      ***
nh       MN  0.0050  0.0004  0.4904  0.7175  1.0000      --       **
         RA  0.0213  0.0124  0.6588  0.9159  1.0000      --      ***
remodnav MN  1.0000  1.0000  0.4904  0.2783  0.0013  0.0014       --
         RA  0.9799  0.9923  0.2135  0.0575  0.0002  0.0005       --

**_f1_**

In [30]:
post_hoc_fix_f1 = sm.post_hoc_table(sdt_nemenyi, f"fixation_{peyes.constants.F1_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_fix_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.       *       *     n.s.
         RA      --    n.s.    n.s.    n.s.       *       *     n.s.
ivvt     MN  0.8071      --       *       *     ***     ***     n.s.
         RA  0.8166      --       *       *     ***     ***     n.s.
idt      MN  0.6886  0.0320      --    n.s.    n.s.    n.s.     n.s.
         RA  0.5910  0.0206      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.5092  0.0124  1.0000      --    n.s.    n.s.     n.s.
         RA  0.4717  0.0108  1.0000      --    n.s.    n.s.     n.s.
engbert  MN  0.0283  0.0000  0.7874  0.9068      --    n.s.       **
         RA  0.0330  0.0001  0.8795  0.9374      --    n.s.       **
nh       MN  0.0227  0.0000  0.7508  0.8831  1.0000      --       **
         RA  0.0485  0.0001  0.9217  0.9637  1.0000      --       **
remodnav MN  0.9918  0.9945  0.2178  0.1155  0.0014  0.0010       --
         RA  0.9864  0.9975  0.1300  0.0815  0.0012  0.0021       --

## Figures
### Agreement Figure (not in article)

In [31]:
W, H = 600, 450

global_metrics_fig = sm.global_metrics_distributions_figure(
    global_measures,
    gt1=GT1, gt2=GT2,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    only_box=False,
    show_other_gt=True,
    share_x=True,
)

global_metrics_fig.update_traces(width=0.9)     # make violins wider so there's less space between them
# Make "Other GT" violins double-sided
for t in global_metrics_fig.data:
    if t["legendgroup"] != "Other GT":
        continue
    t["visible"] = t["name"].split(',')[0] == GT1
    t["side"] = None

global_metrics_fig.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, range=[0, 1], tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, range=[0, 1], tickfont=dict(size=14)),
    yaxis3=dict(showgrid=False, zeroline=False, showline=False, range=[0, 1], tickfont=dict(size=14)),
    xaxis3=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=10, pad=0),
    showlegend=False,
)
# global_metrics_fig.layout.annotations = []    # remove subtitles

# FIG_ID, IS_SUPP = 3, False
# save_fig(global_metrics_fig, FIG_ID, '', 'sample-global-metrics', IS_SUPP)
global_metrics_fig.show()

### Sensitivity Figure (not in article)

In [32]:
W, H = 750, 400

sdt_metrics_fig = sm.sdt_distributions_figure(
    fixation_sdt,
    GT1, GT2,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    only_box=False,
    show_other_gt=True,
    share_x=True,
)
sdt_metrics_fig.update_traces(width=0.9)     # make violins wider so there's less space between them

sdt_metrics_fig.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',

    # remove axis grids
    xaxis=dict(showgrid=False, zeroline=False, showline=False), yaxis=dict(showgrid=False, zeroline=False, showline=False),
    xaxis2=dict(showgrid=False, zeroline=False, showline=False), yaxis2=dict(showgrid=False, zeroline=False, showline=False),
    xaxis3=dict(showgrid=False, zeroline=False, showline=False), yaxis3=dict(showgrid=False, zeroline=False, showline=False),
    xaxis4=dict(showgrid=False, zeroline=False, showline=False), yaxis4=dict(showgrid=False, zeroline=False, showline=False),

    # move legend to bottom
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='container', yref='container', x=0.5, y=0.05),
    showlegend=False,   # hide legend
    margin=dict(l=40, r=0, b=0, t=20, pad=0),
)

# move annotations to the top/left of the plot
# sdt_metrics_fig.for_each_annotation(lambda ann: ann.update(x=-0.05, textangle=-90) if ann.text in row_titles else ann.update(y=0.99))
# sdt_metrics_fig.layout.annotations = []    # remove subtitles

# FIG_ID, IS_SUPP = 8, True
# save_fig(sdt_metrics_fig, FIG_ID, "", f"detection-sample_level", IS_SUPP)
sdt_metrics_fig.show()

### Article Figure (Ridge plots)

In [62]:
GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=16, color=FONT_COLOR)
AXIS_LABEL_STANDOFF = 2

WIDTH, HEIGHT = 900, 1000
ROW_TITLES = ["Cohen's Kappa", "MCC", "1-<i>NLD</i>", "Fixation <i>d'</i>"]
COL_TITLES = [GT1, GT2]

In [78]:
def convert_violin_to_ridge(tr: go.Trace) -> go.Trace:
    new_trace = copy.deepcopy(tr)
    # change values across all traces
    new_trace["visible"] = True
    new_trace["side"] = "positive"
    new_trace["opacity"] = 0.95
    new_trace["width"] = 1.8
    new_trace["box"] = None
    new_trace["showlegend"] = False
    # convert to ridge plot
    if new_trace['x0'] == "Other GT":
        new_trace['y0'] = new_trace["name"] = new_trace["legendgroup"] = "2<sup>nd</sup> Ann."
    elif new_trace["x0"].startswith("i"):
        new_trace['y0'] = new_trace["name"] = new_trace["legendgroup"] = new_trace["x0"].replace("i", "I-").upper()
    elif new_trace["x0"] == "remodnav":
        new_trace["y0"] = new_trace["name"] = new_trace["legendgroup"] = "REMoDNaV"
    else:
        new_trace["y0"] = new_trace["name"] = new_trace["legendgroup"] = new_trace["x0"].upper()
    new_trace['x'] = new_trace['y']
    new_trace['y'] = new_trace['x0'] = None
    new_trace["meanline"] = dict(visible=True, width=3, color='lightgray')
    return new_trace

In [79]:
fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    row_titles=ROW_TITLES, column_titles=COL_TITLES,
    shared_xaxes=False, shared_yaxes=True,
    vertical_spacing=0.06, horizontal_spacing=0.02,
)

# AGREEMENT MEASURES
for tr in global_metrics_fig.data:
    new_tr = convert_violin_to_ridge(tr)
    gt_name = tr["name"].split(',')[0]
    col = [GT1, GT2].index(gt_name) + 1
    row = [peyes.constants.COHENS_KAPPA_STR, peyes.constants.MCC_STR, peyes.constants.COMPLEMENT_NLD_STR].index(tr["scalegroup"]) + 1
    new_tr["showlegend"] = row == 0 and col == 0
    fig.add_trace(new_tr, row=row, col=col)

# SENSITIVITY MEASURES
for tr in sdt_metrics_fig.data:
    if not tr["scalegroup"].endswith("d_prime"):
        # ignore all traces that aren't d'
        continue
    new_tr = convert_violin_to_ridge(tr)
    gt_name = tr["name"].split(',')[0]
    col = [GT1, GT2].index(gt_name) + 1
    fig.add_trace(new_tr, row=len(ROW_TITLES), col=col)

# UPDATE LAYOUT
# update axes
fig.for_each_xaxis(lambda xax: xax.update(
    showline=False,
    zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    range=[-0.025, 1.025] if xax['anchor'] not in ['y7', 'y8'] else None,
    tickfont=AXIS_TICK_FONT,
))
fig.for_each_yaxis(lambda yax: yax.update(
    title=dict(
        text="Detector" if yax['anchor'] in ['x', 'x3', 'x5', 'x7'] else "", font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF
    ),
    showline=False,
    zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
    tickfont=AXIS_TICK_FONT,
))

# update titles
fig.for_each_annotation(lambda ann: ann.update(
    font=TITLE_FONT if ann.text in COL_TITLES else AXIS_LABEL_FONT,
    xref='paper', xanchor='center',
    yref='paper', yanchor='top',
    textangle=0,
))
subtitle_ycoords = [0.78, 0.51, 0.25, -0.025]
for ann in fig.layout.annotations:
    if ann.text in COL_TITLES:
        ann['x'] = 0.25 if ann.text == COL_TITLES[0] else 0.75
        ann['y'] = 1.025
    elif ann.text in ROW_TITLES:
        ann['x'] = 0.49
        ann['y'] = subtitle_ycoords[ROW_TITLES.index(ann.text)]
    else:
        print(f"Unknown annotation: {ann.text}")

# update figure layout
fig.update_layout(
    font_family=FONT_FAMILY,
    width=WIDTH, height=HEIGHT,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=15, b=45, pad=0),
    showlegend=False,
)

NAME = "fig1-sample_agreement"
fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
fig.show()